In [39]:
import csv, time, json, ast
from neo4j import GraphDatabase

In [40]:
startTime = time.time()
song_list = set()
genre_list = set()
artist_list = set()
group_list = set()
P = set()
with open('../kg_data/music_kg.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    count = 0
    for row in reader:
        if row[0] == 's':
            continue
        P.add(row[1])
        if row[2] == 'http://inf558.org/mynamespace#artist':
            artist_list.add(row[0])
        if row[2] == 'http://inf558.org/mynamespace#group':
            group_list.add(row[0])
        if row[2] == 'http://inf558.org/mynamespace#song':
            song_list.add(row[0])
        if row[2] == 'http://inf558.org/mynamespace#genre':
            genre_list.add(row[0])
    f.close()

    song_list = [*song_list]
    genre_list = [*genre_list]
    artist_list = [*artist_list]
    group_list = [*group_list]
    connector = GraphDatabase.driver(uri = 'bolt://localhost:11003', auth=('neo4j', 'DSCI558'))
    session = connector.session()

In [41]:
P

{'acousticness',
 'active_years',
 'aliases',
 'belongs_to',
 'born_in',
 'composed_by',
 'danceability',
 'discription',
 'energy',
 'http://xmlns.com/foaf/0.1/name',
 'instrumentalness',
 'loudness',
 'member_of',
 'performed_by',
 'release_year',
 'subgenre_of',
 'type',
 'valence'}

In [42]:
session.run('MATCH (n) DETACH DELETE n')
query = []
for song_uri in song_list:
    query.append('CREATE (s:Song {uri:' + '"' + song_uri + '"' + '}) RETURN s')
for artist_uri in artist_list:
    query.append('CREATE (a:Artist {uri:' + '"' + artist_uri + '"' + '}) RETURN a')
for group_uri in group_list:
    query.append('CREATE (b:Band {uri:' + '"' + group_uri + '"' + '}) RETURN b')
for genre_uri in genre_list:
    query.append('CREATE (g:Genre {uri:' + '"' + genre_uri + '"' + '}) RETURN g')

In [43]:
with open('../kg_data/music_kg.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if row[1] == 'release_year':
            query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '}) SET s.year = ' + '"' + row[2] + '"' +  ' RETURN s')    
        if row[1] == 'http://xmlns.com/foaf/0.1/name':
            if row[0] in song_list:
                query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '}) SET s.name = ' + '"' + row[2].replace('"', "'") + '"' +  ' RETURN s')    
            if row[0] in artist_list:
                query.append('MATCH (a:Artist {uri:' + '"' + row[0] + '"' + '}) SET a.name = ' + '"' + row[2].replace('"', "'") + '"' +  ' RETURN a')  
            if row[0] in group_list:
                query.append('MATCH (b:Band {uri:' + '"' + row[0] + '"' + '}) SET b.name = ' + '"' + row[2].replace('"', "'") + '"' +  ' RETURN b')  
            if row[0] in genre_list:
                query.append('MATCH (g:Genre {uri:' + '"' + row[0] + '"' + '}) SET g.name = ' + '"' + row[2].replace('"', "'") + '"' +  ' RETURN g')  
        if row[1] == 'aliases':
            query.append('MATCH (a:Artist {uri:' + '"' + row[0] + '"' + '}) SET a.aliases = ' + '"' + row[2].replace('"', "'") + '"' +  ' RETURN a')    
        if row[1] == 'valence':
            query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '})' + ' SET s.valence = {} RETURN s'.format(row[2])) 
        if row[1] == 'loudness':
            query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '})' + ' SET s.loudness = {} RETURN s'.format(row[2])) 
        if row[1] == 'acousticness':
            query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '})' + ' SET s.acousticness = {} RETURN s'.format(row[2]))  
        if row[1] == 'energy':
            query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '})' + ' SET s.energy = {} RETURN s'.format(row[2]))  
        if row[1] == 'danceability':
            query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '})' + ' SET s.danceability = {} RETURN s'.format(row[2]))  
            
        if row[1] == 'belongs_to':
            query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '}) MATCH (g:Genre {uri:' + '"' + row[2] + '"' + '}) CREATE (s)-[:BELONGS_TO]->(g)')  
            
        if row[1] == 'born_in':
            if row[0] in artist_list:
                query.append('MATCH (a:Artist {uri:' + '"' + row[0] + '"' + '}) SET a.born_in = ' + '"' + row[2] + '"' +  ' RETURN a')  
            if row[0] in group_list:
                query.append('MATCH (b:Band {uri:' + '"' + row[0] + '"' + '}) SET b.born_in = ' + '"' + row[2] + '"' +  ' RETURN b') 
                
        if row[1] == 'active_years':
            if row[0] in artist_list:
                query.append('MATCH (a:Artist {uri:' + '"' + row[0] + '"' + '}) SET a.active_years = ' + '"' + row[2] + '"' +  ' RETURN a')  
            if row[0] in group_list:
                query.append('MATCH (b:Band {uri:' + '"' + row[0] + '"' + '}) SET b.active_years = ' + '"' + row[2] + '"' +  ' RETURN b') 
                
        if row[1] == 'composed_by':
            if row[2] in artist_list:
                query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '}) MATCH (a:Artist {uri:' + '"' + row[2] + '"' + '}) CREATE (s)-[:COMPOSED_BY]->(a)')
            if row[2] in group_list:
                query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '}) MATCH (b:Band {uri:' + '"' + row[2] + '"' + '}) CREATE (s)-[:COMPOSED_BY]->(b)') 
                
        if row[1] == 'performed_by':
            if row[2] in artist_list:
                query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '}) MATCH (a:Artist {uri:' + '"' + row[2] + '"' + '}) CREATE (s)-[:PERFORMED_BY]->(a)')
            if row[2] in group_list:
                query.append('MATCH (s:Song {uri:' + '"' + row[0] + '"' + '}) MATCH (b:Band {uri:' + '"' + row[2] + '"' + '}) CREATE (s)-[:PERFORMED_BY]->(b)') 
                
        if row[1] == 'discription':
            if row[0] in artist_list:
                query.append('MATCH (a:Artist {uri:' + '"' + row[0] + '"' + '}) SET a.discription = ' + '"' + row[2].replace('"', "'") + '"' +  ' RETURN a')  
            if row[0] in group_list:
                query.append('MATCH (b:Band {uri:' + '"' + row[0] + '"' + '}) SET b.discription = ' + '"' + row[2].replace('"', "'") + '"' +  ' RETURN b')  
            if row[0] in genre_list:
                query.append('MATCH (g:Genre {uri:' + '"' + row[0] + '"' + '}) SET g.discription = ' + '"' + row[2].replace('"', "'") + '"' +  ' RETURN g')
                
        if row[1] == 'member_of':
            query.append('MATCH (a:Artist {uri:' + '"' + row[0] + '"' + '}) MATCH (b:Band {uri:' + '"' + row[2] + '"' + '}) CREATE (a)-[:MEMBER_OF]->(b)')
            
        if row[1] == 'subgenre_of':
            query.append('MATCH (g1:Genre {uri:' + '"' + row[0] + '"' + '}) MATCH (g2:Genre {uri:' + '"' + row[2] + '"' + '}) CREATE (g1)-[:SUBGENRE_OF]->(g2)')
            


In [1]:
# for line in query:
#     session.run(line)